In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import math
import time
from matplotlib import pyplot as plt

In [2]:
# Initialize Global Variables
EPOCHS = 1000
LAYERSIZE = 40
NUM_MODELS = 10

In [3]:
# Initialize Dataset

# Read .csv files for training

xdata = pd.read_csv('ML_inputs_tear.csv')
ydata = pd.read_csv('U_tear.csv').T

# Adding Constant Parameters to xdata
xdata.insert(0, 'c', 6)
xdata.insert(1, 'D', .05)
xdata.insert(2, 'k1', 15)
xdata.insert(3, 'k2', 15)
xdata.insert(4, 'N', 2)
xdata.insert(5, 'kappa_1', .25)
xdata.insert(6, 'kappa_2', .25)
xdata.insert(7, 'kappa_3', .25)

# Switch to numpy array
x = xdata.to_numpy()
y = ydata.to_numpy()


In [4]:
# Import Models

annmodels = []

for i in range(NUM_MODELS):
    annmodels.append(tf.keras.models.load_model('model' + str(i) + '.h5'))

In [5]:
# Timing Calculations
modelTime = []

for model in annmodels:
    
    # Perform 50 timed predictions, average result
    tmpTime = []
    for i in range(50):
        start = time.time()
        model.predict(x)
        end = time.time()
        tmpTime.append(end - start)
        
    modelTime.append(np.mean(tmpTime))

8/8 [==============================] - 0s 3ms/step


8/8 [==============================] - 0s 2ms/step


8/8 [==============================] - 0s 3ms/step


8/8 [==============================] - 0s 3ms/step


In [6]:
# Error Calculations
modelError = []

for model in annmodels:
    # find the relative frobenius error of the prediction
    pred = model.predict(x)
    
    toterr = 0
    for i in range(len(x)):
        Norm1 = np.linalg.norm(y[i] - pred[i], 2)
        Norm2 = np.linalg.norm(pred[i], 2)
        toterr = toterr + ((Norm1**2)/(Norm2**2))
        
    modelError.append(math.sqrt(toterr))

8/8 [==============================] - 0s 3ms/step


In [7]:
export = pd.DataFrame({'online time': modelTime, 'error': modelError})
offlineTime = pd.read_csv('offlineTime.csv')

export = pd.concat((offlineTime, export), axis = 1)
FileName = 'layer'+str(LAYERSIZE)+'epoch'+str(EPOCHS)+'.csv'
export.to_csv(FileName, encoding = 'utf-8', index=False)
#export.to_csv('MLModelData.csv', encoding = 'utf-8', index=False)
export

#export = pd.DataFrame({'offline time': totaltime,'online time': ontime, 'error':l2err})
#FileName = 'layer'+str(LAYERSIZE)+'epoch'+str(EPOCHS)+'.csv'
#export.to_csv(FileName, encoding = 'utf-8', index=False)

,offline time,online time,error
0,53.852038,0.103644,0.076822
1,52.853822,0.101980,0.063212
2,53.539997,0.098171,0.144875
3,52.431435,0.096928,0.085361
4,52.419984,0.101847,0.064717
5,49.358641,0.100121,0.071850
6,52.419484,0.096448,0.110890
7,49.605522,0.097964,0.074601
8,51.868109,0.095919,0.102784
9,51.415905,0.098871,0.094888


In [8]:
L10 = pd.read_csv('layer10epoch650.csv')
L40 = pd.read_csv('layer40epoch650.csv')
L100 = pd.read_csv('layer100epoch650.csv')
L1000 = pd.read_csv('layer1000epoch650.csv')

E50 = pd.read_csv('layer40epoch50.csv') 
E100 = pd.read_csv('layer40epoch100.csv')
E300 = pd.read_csv('layer40epoch300.csv')
E1000 = pd.read_csv('layer40epoch1000.csv')

LVary = pd.concat((L10, L40, L100, L1000), axis = 1)
EVary = pd.concat((E50, E300, L40, E1000), axis = 1)

LVary.to_csv('layerVary.csv', encoding='utf-8', index=False)
EVary.to_csv('epochVary.csv', encoding='utf-8', index=False)